# Manipulating Data & Data Frames

Last week we did some basic EDA (Exploratory Data Analysis) with the MetOffice weather data, but it was limited to mainly looking at a few graphs right at the end of the practical. Thanks to that, depending on the data you got you may spotted a 'problem' or two (_e.g._ temperature readings at the start or end of the time series that were basically 0).

This week we want to tackle this in a more systematic way. We are going to switch data sets because the Socio-Economic data is easier to manipulate than the weather data and it has a few features that are particularly useful for demonstrating the value of transformation and standardisation.

## Data Series & Indexing

A DataFrame is composed of one or more data series (columns) objects and an index that is a non-data column useful for finding individual observations. In our 'city data' data set, the index would be the city names themselves because the names _aren't_ data in the usual sense: you can't calculate a mean from them and they aren't categorical variables (e.g. 'Metro' vs 'Town') that we'd use for grouping. They are unique non-data values, so that's your index.

## Creating your own index

Ordinarily, the data making up a df are read directly from a file and the index is automatically built using the first available 'index-like' column in the file. But you are not bound by what pandas thinks is the 'right' thing to do: you can set any column as an index, or even create one of your own!

For instance, let's say that you wanted a series containing only latitudes for British cities, you could create a new Series with this custom index as follows:
```python
myLatitudes = pd.Series(
    [7063197, 6708480, 6703134, 7538620], 
    index = ['Liverpool', 'Bristol', 'Reading', 'Glasgow']
)
```
In this case, the index is a list of cities and it would, generally, make it quite quick to look up the latitude of any of the cities listed. You are never limited to _only_ looking up values by index, but this is usually faster.

In [ ]:
import pandas as pd
myLatitudes = pd.Series(
    [7063197, 6708480, 6703134, 7538620], 
    index = ['Liverpool', 'Bristol', 'Reading', 'Glasgow']
)
print "Type of myLatitudes: "      + str(type(myLatitudes))
print "Access like a dictionary: " + str(myLatitudes['Liverpool'])
print "Access like a method: "     + str(myLatitudes.Liverpool)

myLatitudes.Bristol = '555000'

print "Updated latitude: " + str(myLatitudes.Bristol)

You'll notice that we also just accessed the df in two different ways -- understanding the strengths and weaknesses of these two approaches is really important:

1. The 'method' approach (`<df>.<series name>` and `<df>.<index name>`) makes for code that is easy to read. A good example of that would be the `df.Population.mean()` that we saw above.

2. The 'dictionary' approach (`<df>['<series name>']` and `<df>['<index value>']`) is helpful when there is potentially ambiguity about what you want Python to do (you shouldn't run into this problem very often), but it's mainly about being able to access or modify a _range_ of values... as we'll see below.

## Access Data Using `loc` and `iloc`

So, what about if you wanted to select several values from the df at the same time? How do you select, say, rows in the range from 0 to 2, or select Bristol and Glasgow in one go? 

Here's how:

In [ ]:
# Access like a list
print myLatitudes.iloc[0:2]

print "\n"

# Access a range
print myLatitudes.loc['Reading':]

print "\n"

# Access non-sequential values
print myLatitudes.loc[ ['Bristol','Glasgow'] ]

print "\n"

# Select non-sequential, non-indexed values
# (converts result to a Boolean)
print myLatitudes.isin([6703134, 7538620])

# Now use two _different_ ways to select 
# data for Liverpool, Bristol and Glasgow
print myLatitudes.???

print "\n"

print myLatitudes.???

A simple mnemonic for loc and iloc is that iloc is about using _integers_ (i == integers) to to help you to find something in the data frame (like working with a list), while loc is about using the index location _directly_ in a list-like way.

*Note*: there is a [lot more](http://pandas.pydata.org/pandas-docs/version/0.18.1/indexing.html#selection-by-position) that you can do with this.

*Note*: using loc/ix is also how you '[set with enlargement](http://pandas.pydata.org/pandas-docs/version/0.18.1/indexing.html#setting-with-enlargement)' -- meaning that this is how you have to write a query when you want to append to the data frame in some way (add a row or add a column).

### Where

More complex getting and setting operations can be performed using `where`, but the majority of students will not (yet) need its full power. If, however, you are trying to perform a complex selection or update and cannot figure out how to do so within the constraints of what we've seen, then [`where` is your friend](http://pandas.pydata.org/pandas-docs/version/0.18.1/indexing.html#the-where-method-and-masking).

## Sampling

OF course, sometimes you don't want a particular range of data, you want a _random sample_ so that you can either a) get a better sense of the data, or b) perform some kind of test with a subsample before replicating on the full data set. Pandas has [got you covered](http://pandas.pydata.org/pandas-docs/version/0.18.1/indexing.html#selecting-random-samples) with a huge range of options, including sampling with replacement, sample weights, row numbers and a fraction of the data set. 

Let's look at some simple examples with the full NS-SeC data set:

In [ ]:
df.sample(n=5)[ ['CDU','GeoCode','GeoLabel'] ] # Sample of size 5

In [ ]:
df.sample(n=5)[ ['CDU','GeoCode','GeoLabel'] ] # This will not give you the same sample

In [ ]:
df.sample(frac=0.00025)[ ['CDU','GeoCode','GeoLabel'] ] # Sample a fraction of the rows

### String Formatting for Pretty-Printed Numbers 

Notice also the `<string>.format()` command I’ve used here: `{0:> 8.1f}`. In order to understand how this works for formatting the results in a nice, systematic way you will need to read  [the documentation](http://www.python.org/dev/peps/pep-3101/)

The 'pep' tells you that:
* `{0}` tells Python to grab the first value inside the parentheses (`format(... values ...)`) and to stick it into the string at this point, but `:...` tells Python that we also format the string in a particular way specified in the `...`.
* `>` tells Python that the string should be right-aligned.
* The space (' ') next to the > says that any 'fill' should be done with whitespace (you could also do it with a 0).
* `8.1f` tells Python to treat anything it gets as a float (even if the variable is an int) and to format it for having 1 significant digit after the full-stop, and a total of 8 digits in all (which ties us back to the right-alignment up above). If you give it a number that has more than 3 digits to the left of the full-stop then it will still print them out,.

Here are some suggestions to better-understand what’s going on:
* Try changing the > to a < to see what happens to the alignment (then change it back)
* Try changing the .1 to a .0 to see what happens to the alignment and precision
* Try changing the .0 to a .6 to see what happens to the alignment and precision
Do this in the coding area below.

In [ ]:
print "\tMedian:  {0:???f}".format(series.median())

## Finding Rows in the Data

You should remember from Week 5 that we can find out _where_ the maximum value in the data occurs using this code:
```python
df[df.Group1Pct == df.Group1Pct.max()]
```

Can you figure out how to list all of the LSOAs where the proportion of Group 1 residents is greater than 50%? 

Now, let's find out what are the top 10 areas in terms of the concentration of Group 1 residents and compare to the top 10 areas in terms of _raw_ counts of Group 1 residents... This is a nice illustration of how you can _chain_ together a whole series of methods to do some pretty cool stuff:

In [ ]:
df.sort_values(by='Group1Pct', ascending=False).head(10)[['GeoLabel','Group1','Group1Pct']]

In [ ]:
df.sort_values(by='Group1', ascending=False).head(10)[['GeoLabel','Group1','Group1Pct']]

Just so that you understand what we just did with this:
1. Take the data frame `df`;
2. Sort it by descending order;
3. Take the first ten values;
4. Print out the columns specified by the list.

Let's pull it apart step-by-step at the code level:

* The first step in this process is `df.sort_values` -- you can probably guess what this does: it sorts the data frame!
* The parameters passed to the `sort_values` function are `by`, which is the column on which to sort, and `ascending=False`, which gives us the data frame sorted in _descending_ order!
* The output of `df.sort(...)` is a _new_ data frame, which means that we can simply add `.head(10)` to get the first ten rows of the newly-sorted data frame.
* And the output of `df.sort(...).head(...)` is yet _another_ data frame, which means that we can print out the values of selected columns using the 'dictionary-like' syntax: we use the outer set of square brackets (`[...]`) to tell pandas that we want to access a subset of the top-10 data frame, and we use the inner set of square brackets (`['GeoLabel','Group1','Group1Pct']`) to tell pandas which columns we want to see.

I'd say 'simples, right?' but that's obviously _not_ simple. It _is_, however, very, very _elegant_ because it's quite clear (once you get past the way that lots of methods can be chained together) and it's very succinct (we did all of that in _one_ line of code!).

### Quick Quiz

So, given these results, which do you think is a _better_ answer to the question: where is there the highest concentration of highly-skilled 'NS-SeC Group 1' residents in England and Wales? Were these the answer that you were expecting?

# Taking a Cut of the Data

For some types of plots we can be overwhelmed by the data -- trying to show the distribution of Group 1 people as grouped by, say, borough or Local Authority would be tricky since, not only are there a lot of them, but we also don't even _have_ a LA column to use!

To take a 'cut' of the data based on LAs, we need some way to get to grips with what's happening in the `GeoLabel` series as that's pretty obviously the only place we're going to get location data. 

## Taking a Random Sample

Rather than trying to step through the whole data frame, wouldn't it be handy to take a random sample first. There are two ways to do this; as usual, [Stack Overflow](http://stackoverflow.com/questions/15923826/random-row-selection-in-pandas-dataframe) is your friend, but in this case it turns out that pandas has improved since that question was asked (which is why it always pays to check the latest documentation) and now supports sampling directly:

In [ ]:
# Now we can take a random sample
# of 20 rows from df (or any other
# number of rows)
sdf = df.sample(n=20)

# And let's look at the geography
sdf.GeoLabel

If you run the random sample several times, you'll see that there is something of a patter here: there is something that is obviously the name of a Local Authority (LA) and it's often followed by some kind of identifier (because LSOAs are smaller than LAs). There are a couple of ways that we can approach this:

1. We could try to match on the name of the LA in the GeoLabel (*e.g.* find the GeoLabels starting with 'Hackney')
2. We could try to find a way to strip off the identifier so that what we'd be left with was the name of LA.

### Matching on the Start of a Word

This is easier in the short-term because we can just say "find the Hackney GeoLabels" but it's less flexible in the long-term because we can't actually use the GeoLabel column as a way to group our results (because each GeoLabel is _still_ unique, so grouping will group by LSOA).

But let's take a look at how that works anyway:

In [ ]:
df.ix[df.GeoLabel.str.startswith('Hackney'),['CDU','GeoCode','GeoLabel','Total']].head(10)

You'll see that that gave us the Hackney LSOAs, but without modifying any of the data. This is also the first time you've seen the `<data frame>.ix` notation which allows us to combine row and and column access via a mix of integer and label. In other words, you can use this to [select _any_ kind of subset you like](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.ix.html).

Just to make it really obvious what I did:
```python
<data frame>.ix[ <row selection criteria>, <column selection criteria> ].head()
```
So this is:
1. Select rows where `<data frame>.<series name>.str.startswith(<search string>)` (treats the LSOA GeoLable as a string and the searches for strings that start with...)
2. Select columns in the list `['CDU','GeoCode','GeoLabel','Total']`
3. Return the first 10 rows using `head()`

You could _also_ use integer selection on the columns as you would with any normal list: e.g. `xrange(2,5)`.

### Extracting the LA* Name

* _[Note: technically this isn't a local authority, but it's close enough for our purposes.]_

The alternative approach, actually extracting the name of LA to a new column, requires a more sophisticated approach because we need to define a _pattern_ that takes into account the different ways that the LA names are embedded in the LSOA GeoLabel columns. For that, we need something called a regular expression. 

Regular expressions are the real workhorses of data munging: we can use them to set up complex patterns that split, remove, replace, or change text columns in our data set. They are well beyond the scope of Term 1, but it's important that you know that: a) they exist, and b) you'll have to cope with them at some point.

After poking around in the data it seems there are generally four different ways that LAs have additional numbers/letters that distinguish between the sub-regions inside each LA:
* 3 numbers and a letter: _e.g._ ' 060D'
* A hyphen and some numbers: _e.g._ ' - 03'
* An underscore and a number: _e.g._ '\_1'
* Nothing: _e.g._ [none] or ''

Each of these has a matching regular expression that allows us to find it in a string:
* `\s\d+[A-Z]` = find a space (`\s`), followed by 1 or more numbers (`\d+`), and a capital letter (`[A-Z]`)
* `\s-\s\d+` = find a space (`\s`), followed by a hyphen, followed by another space (`\s`), and then one or more numbers (`\d+`)
* `_\d+` = find an underscore followed by one or more numbers (`\d+`)

You'll have to [look up how to use regular expressions](https://developers.google.com/edu/python/regular-expressions) to make sense of the rest, but here's how we do it the _hard way_ that is also the more flexible way:

In [ ]:
df['LA'] = df.GeoLabel.str.extract('(^[^_\d]+)(?:_\d+|\s\d+[A-Z]|\s-\s\d+)?$', expand=False)
df['LA'] = df['LA'].astype('category')

In [ ]:
# Don't believe me? Take a look.
some(df, 10)[ ['GeoLabel','LA'] ]

### Selecting by Match

Now we can take a 'cut' of the data by selecting only some London-based boroughs for further analysis... we’re going to arbitrarily select K&C, Hackney and Barking because I know they’re quite different boroughs, but you could also use the data to make this selection, right? For instance, I could ask pandas to help me pick the three boroughs that are the furthest apart in terms of their Group 1 means...

Anyway, here’s the code to select a cut:
```python
# sdf = subsetted data frame

# Select where the LA value 'is in' one of our pre-defined list
sdf = df.loc[df.LA.isin(['Kensington and Chelsea','Hackney','Barking and Dagenham'])]

# Remove the remaining unused categories
sdf.LA = sdf.LA.cat.remove_unused_categories()

# And a simple check to see how many categorical values are left
print("sdf now contains: {0} values".format(sdf.LA.describe().unique()[1]))
```
You’ll notice two unusual bits of code in there that need some explanation:
* To 'select  multiple' we need to write: `<data frame>.<series name>.isin([...])`, so we can't just write `<data frame>.<series name>==[...]` unfortunately.
* We also have this line: `<data frame>.<column name> = <data frame>.<column name>.cat.remove_unused_categories()`. This should be fairly self-explanatory, but it’s because by default pandas doesn’t update the list of valid categories (i.e. Local Authorities) just because we filtered out boroughs that weren’t of interest. We therefore need to update the Series so that Seaborn doesn’t include a bunch of empty categories in when we make our plots.

#### Over to you!

Now make use of this code to produce overlay plots (i.e. show on the same plot the data for all three boroughs) for each of the NS-SeC Groups in the data. You should use a for loop to do this so that the group is dynamically specified (i.e. each series is copied to a variable, not hard-coded) and all of the outputs are formatted the same way! I'd suggest trying with one group first to get things working and _then_ doing it for all eight groups.

In other words:
```python
for g in ['Group1']:
    ???
```

Get coding!